## 1. 임포트

In [ ]:
# import 및 환경변수 설정
import os
import openai
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 지정된 데이터 불러옴
from test_data import DOG_PROFILES, QUESTIONS

# openai 키
openai.api_key =""

In [ ]:
# 테스트용 (2 * 4set 8회 테스트)

DOG_PROFILES=[
    {
        "dog_name": "코코",
        "dog_breed": "말티즈",
        "dog_age": 3,
        "dog_gender": "여아",
        "dog_neutered": "중성화 완료",
        "dog_disease": "없음",
        "dog_disease_detail": "",
        "dog_period": "② 1년 이상~3년 미만",
        "dog_housing": "아파트"
    },
    {
        "dog_name": "보리",
        "dog_breed": "푸들",
        "dog_age": 2,
        "dog_gender": "남아",
        "dog_neutered": "중성화 미완료",
        "dog_disease": "없음",
        "dog_disease_detail": "",
        "dog_period": "① 1년 미만",
        "dog_housing": "아파트"
    }
]

QUESTIONS = [
    '식사 후 구토를 자주 해요. 위장이 약한 걸까요?', 
    '요즘 문만 열면 밖으로 도망가려고 해요. 스트레스 때문일까요?', 
    '요즘 발바닥을 자꾸 핥아요. 발에 이상 있는 걸까요?',
    '배변을 너무 자주 해요. 하루에 5번은 넘는 것 같은데 정상일까요?', 
]

## 2. 프로필 프롬프트

In [2]:
def profile_to_text(profile: dict) -> str:
    """빈 값은 생략하여 반려견 정보를 자연스럽게 텍스트로 구성"""
    fields = {
        "이름": profile.get("dog_name"),
        "견종": profile.get("dog_breed"),
        "나이": f"{profile.get('dog_age')}세" if profile.get("dog_age") else None,
        "성별": profile.get("dog_gender"),
        "중성화 여부": profile.get("dog_neutered"),
        "질병": profile.get("dog_disease"),
        "세부 질병": profile.get("dog_disease_detail"),
        "보호기간": profile.get("dog_period"),
        "주거형태": profile.get("dog_housing")
    }
    return ", ".join(f"{k}: {v}" for k, v in fields.items() if v)


In [3]:
profile_to_text(DOG_PROFILES[1])

'이름: 보리, 견종: 푸들, 나이: 2세, 성별: 남아, 중성화 여부: 중성화 미완료, 질병: 없음, 보호기간: ① 1년 미만, 주거형태: 아파트'

## 3. 모델 응답 함수

In [4]:
MODEL_NAME = "gpt-4"
TEMPERATURE = 0.7

def get_openai_response(prompt: str) -> str:
    response = openai.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=TEMPERATURE
    )
    return response.choices[0].message.content.strip()



## 4. 테스트 실행 반복

In [ ]:
results = []

for idx, profile in enumerate(DOG_PROFILES, 1):
    profile_text = profile_to_text(profile)

    for question in QUESTIONS:
        prompt = f"반려견 정보: {profile_text}\n질문: {question}\n답변:"
        start = time.time()
        response = get_openai_response(prompt)
        end = time.time()

        elapsed = round(end - start, 3)
        response_length = len(response)

        results.append({
            "profile_ID": idx,
            "dog_profile": profile_text,
            "question": question,
            "response": response,
            "time": elapsed,
            "length": response_length,
            "speed_per_sec": round(response_length / elapsed, 2) if elapsed > 0 else 0,
            "speed_per_text": round(elapsed / response_length, 4) if response_length > 0 else 0,
        })
        
        # 진행 확인
        print(f'{idx} | 응답완료 | 질문 : {question}')


## 5. 결과 저장

In [6]:
df = pd.DataFrame(results)
df.to_csv("./results/openai_results.csv", index=False, encoding="utf-8-sig")